## Kalp Rahatsızlığı Analizi


<img src="http://idora.gazetevatan.com/vatanmediafile/Haber598x362/2018/12/06/kalp-sagliginizla-ilgili-bilmeniz-gereken-7-gercek-3879416.Jpeg" width="700px">

In [ ]:
# for basic operations
import numpy as np
import pandas as pd
import pandas_profiling

# for data visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for advanced visualizations 
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected = True)


# for providing path
import os
print(os.listdir('../input/'))

# for model explanation
import shap

## Çalışma Metodolijisi
1.  [Veri Yükleme ve Veri Tanıma](#1)
1.  [Veri Görselleştirme ](#2)
1.  [Veri Hazırlama ve Train/Test ](#3)
1.  [Modelleme - KNN ](#4)


<a id="1"></a><br>
# Veri Yükleme ve Tanıma 

In [ ]:
# reading the data
data = pd.read_csv('../input/heart.csv')

# getting the shape
data.shape

<a id="1"></a><br>
# Veri Yükleme ve Tanıma 

age: The person's age in years


sex: The person's sex (1 = male, 0 = female)


cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)


trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)


chol: The person's cholesterol measurement in mg/dl


fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)


restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)


thalach: The person's maximum heart rate achieved


exang: Exercise induced angina (1 = yes; 0 = no)


oldpeak: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot. See more here)


slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)


ca: The number of major vessels (0-3)


thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)


target: Heart disease (0 = no, 1 = yes)


In [ ]:
data.head(10)
#veri tanıma amaçlı ilk 10 değerin gözlemlenmesi

In [ ]:
data.describe().T

In [ ]:
data.info()
# 1 adet float  13 adet int dan oluşan veri setine sahibiz ve boş gözlemimiz bulunmamakta

### Veri Profili

In [ ]:
profile=pandas_profiling.ProfileReport(data)
profile
#veri proflinin gözlemlenmesi

<a id="2"></a><br>
## Veri Görselleştirme

In [ ]:
data.isnull().sum()
#null değer var mı var ise adet bazında belirlenmesi

In [ ]:
p= data.hist(figsize=(15,15))
#değişkenler için histogramlar

In [ ]:

g= sns.catplot(x="target" , y="age", data=data,
              height=6, kind="bar", palette="muted")
#standart sapma düşük olmakla birlikte kalp rahatsızlığı olan gözlemlerin ortalama yaş değerleri daha düşüktür.Genel yaş ortalaması 60 ile 50 arasındadır.

In [ ]:
g= sns.catplot(x="target" , y="thalach", data=data,
              height=6, kind="bar", palette="muted")
#Thalach değerlerinee bakılacak olursa kalp rahatsızlığı olanların görece değerlerinin yüksel olduğu görülmektedir.

In [ ]:
g= sns.catplot(x="target" , y="chol", data=data,
              height=6, kind="bar", palette="muted")
#Kolestrol değerleri her iki durum içinde birbirine yakın çıkmıştır.

In [ ]:
#cp değerleri anlamlı bir anlaşma sağlamaktadır.
g= sns.catplot(x="target" , y="cp", data=data,
              height=6, kind="bar", palette="muted")

In [ ]:
# exang değerlerine bakıldığında hedef değişken bazında anlamlı bir ayrıştırma sağlatyabileceği gözükmektedir.
g=sns.catplot(x="target", y="exang", data=data,
             height=6 , kind="bar",  palette="muted" )

In [ ]:
# trestbps değerine bakıldığında değerlerin target değişkenken bazında yakın değerler verdiği gözlenlenmişttir.
g = sns.catplot(x="target" , y="trestbps", data=data,
               height=6 , kind="bar" )

In [ ]:
#uç değer tespit amaçlı boxplot
sz=(9,9)
fig,ax = plt.subplots(figsize=sz)
sns.boxplot(ax=ax, data=data, orient= "h")



In [ ]:
#uç değer bulundu
data2=data["chol"].max()
data2

In [ ]:
#chol uç değerine sahip index tespit edildi.
data1=data[data["chol"]==564]
data1

In [ ]:
#veri yedeklendi
data_yedek=data
data_yedek.info()

In [ ]:
data.drop(index=85,inplace = True)
#Aşırı değere sahip index çıkarıldı

In [ ]:
data.info()

In [ ]:
p=sns.pairplot(data, hue="target")


In [ ]:
#korelasyon tespiti amaçlı headmap kullanıldı

plt.figure(figsize=(12,10))
p=sns.heatmap(data.corr(),annot=True,cmap="RdYlGn")
#en yüksek korelasyon exang ,cp ve oldpeak ile en düşük korelasyon fb değişkeniyledir.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
plt.style.use("fivethirtyeight")
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['age'], color = 'cyan')
plt.title('Distribution of Age', fontsize = 20)
plt.show()
#normal dağılıma yakın bir görüntü sergilemektedir. Veri yaş dağılımı 50-60 yaş arası yoğumlaşmaktadır.

In [ ]:
# checking the distribution of age amonng the patients

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['chol'], color = 'cyan')
plt.title('Distribution of chol', fontsize = 20)
plt.show()
#normal dağılmış bir veri olmasına karşın uç değer bulundurmuş olma ihtimali mevcuttur.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['thalach'], color = 'cyan')
plt.title('Distribution of thalach', fontsize = 20)
plt.show()
#sağa eğik dağılım özelliği göstermiştir.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['trestbps'], color = 'cyan')
plt.title('Distribution of trestbps', fontsize = 20)
plt.show()

In [ ]:
# plotting a donut chart for visualizing each of the recruitment channel's share

size = data['sex'].value_counts()
colors = ['lightblue', 'lightgreen']
labels = "Male", "Female"
explode = [0, 0.01]

my_circle = plt.Circle((0, 0), 0.7, color = 'white')

plt.rcParams['figure.figsize'] = (9, 9)
plt.pie(size, colors = colors, labels = labels, shadow = True, explode = explode, autopct = '%.2f%%')
plt.title('Distribution of Gender', fontsize = 20)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.legend()
plt.show()
#erkek bireylerin ağırlıklı olduğu bir veri setidir. Erkekler %68,32 , Kadınlar %31,68 oluşturmaktadır.

In [ ]:
size = data['target'].value_counts()
colors = ['lightblue', 'lightgreen']
labels = "0", "1"
explode = [0, 0.01]

my_circle = plt.Circle((0, 0), 0.7, color = 'white')

plt.rcParams['figure.figsize'] = (9, 9)
plt.pie(size, colors = colors, labels = labels, shadow = True, explode = explode, autopct = '%.2f%%')
plt.title('Distribution of Hearth Disaster', fontsize = 20)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.legend()
plt.show()
#verimizin %45,54 ü kalp rahatsızlığı olan bireylerden oluşmaktadır.

In [ ]:
# tresbps vs target

plt.rcParams['figure.figsize'] = (12, 9)
sns.boxplot(data['target'], data['trestbps'], palette = 'viridis')
plt.title('Relation of tresbps with target', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
sns.boxplot(data['target'], data['chol'], palette = 'viridis')
plt.title('Relation of chol with target', fontsize = 20)
plt.show()

In [ ]:
trace = go.Scatter3d(
    x = data['target'],
    y = data['oldpeak'],
    z = data['age'],
    name = 'Marvel',
    mode = 'markers',
    marker = dict(
         size = 10,
         color = data['age']
    )
)

df = [trace]

layout = go.Layout(
    title = 'Cholestrol vs Heart Rate vs Age',
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0  
    ),
    scene = dict(
            xaxis = dict(title  = 'target'),
            yaxis = dict(title  = 'oldpeak'),
            zaxis = dict(title  = 'age')
        )
    
)
fig = go.Figure(data = df, layout=layout)
py.iplot(fig)
# 3 boyutlu olarak yaş- oldpeak değerlerinin dağılımları

<a id="3"></a><br>
# Veri Hazırlama ve Train/Test 

In [ ]:
data.head()

In [ ]:
y= data.target.values
x = data.drop(["target"], axis=1)



In [ ]:
y

In [ ]:
#Train - Test ayrımı
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=1/3,random_state=44)


<a id="4"></a><br>
# Modelleme

### KNN

In [ ]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, y_train)
prediction = knn.predict(X_test)
print("{} nn  score : {}".format(3,knn.score(X_test,y_test)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


test_scores = []
train_scores = []

for i in range(1,15):

    knn = KNeighborsClassifier(i)
    knn.fit(X_train,y_train)
    
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))

In [ ]:
## score that comes from testing on the same datapoints that were used for training
max_train_score = max(train_scores)
train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
print('Max train score {} % and k = {}'.format(max_train_score*100,list(map(lambda x: x+1, train_scores_ind))))

In [ ]:
## score that comes from testing on the same datapoints that were used for training
max_test_score = max(test_scores)
test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
print('Max test score {} % and k = {}'.format(max_test_score*100,list(map(lambda x: x+1, test_scores_ind))))

In [ ]:
plt.figure(figsize=(12,5))
p = sns.lineplot(range(1,15),train_scores,marker='*',label='Train Score')
p = sns.lineplot(range(1,15),test_scores,marker='o',label='Test Score')

### SVM

In [ ]:
from sklearn.svm import SVC
 
svm = SVC(random_state = 1)
svm.fit(X_train,y_train)
 


In [ ]:

 print("print accuracy of svm algo: ",svm.score(X_test,y_test))

### Naive Bayes

In [ ]:
 # Naive bayes 
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train,y_train)
 
 # %% test
 print("print accuracy of naive bayes algo: ",nb.score(X_test,y_test))

### Decission Tree

In [ ]:
#%%
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

print("score: ", dt.score(X_test,y_test))

### Random Forest

In [ ]:
#%%  random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100,random_state = 44)
rf.fit(X_train,y_train)
print("random forest algo result: ",rf.score(X_test,y_test))


In [ ]:
from sklearn.ensemble import RandomForestClassifier


test_scores_rf = []
train_scores_rf = []

for i in range(70,120):

    rf = RandomForestClassifier(i,random_state=44)
    rf.fit(X_train,y_train)
    
    train_scores_rf.append(rf.score(X_train,y_train))
    test_scores_rf.append(rf.score(X_test,y_test))

In [ ]:
test_scores_rf
#maksimumskor 90

In [ ]:
plt.figure(figsize=(12,5))

p = sns.lineplot(range(70,120),test_scores_rf,marker='o',label='Test Score')

In [ ]:
#%%  random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 90,random_state = 44)
rf.fit(X_train,y_train)
print("random forest algo result: ",rf.score(X_test,y_test))


#### SVM Accuracy :  0.5148514851485149
#### Decission Tree Accuracy : 0.6534653465346535
#### Random Forest Accuracy:  0.8712871287128713
#### Naive Bays Accuracy : 0.8910891089108911
#### KNN Accuracy : 0.7326732673267327

#### En iyi test score veren Naive Bays kabul edilmiştir.






In [ ]:
y_pred = nb.predict(X_test)
y_true = y_test
#%% confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true,y_pred)


# %% cm visualization
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize =(5,5))
sns.heatmap(cm,annot = True,linewidths=0.5,linecolor="red",fmt = ".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()


#### Kalp rahatsızlığı olan 49 gözlemden 43'ü doğru tahmin edilirkem 6 tanesi yanlış sınıflandırılmıştır.
#### Kalp rahatsızlığı olmayan 53 gözlemden 47'si doğru tahmin edilirkem 5 tanesi yanlış sınıflandırılmıştır.